In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [1]:
train = pd.read_csv("data/train.csv")

test = pd.read_csv("data/test.csv")

sample = pd.read_csv("data/sample_submission.csv")


# 카테고리 타입 변형

In [48]:
category_col = train.columns.difference(['id', 'Age','Flight Distrance','Departure Delay in Minutes', 'Arrival Delay in Minutes'])

for i in train.columns.difference(['id', 'Age','target','Flight Distrance','Departure Delay in Minutes', 'Arrival Delay in Minutes']):
    train['{}'.format(i)] = train['{}'.format(i)].astype('category')

for i in test.columns.difference(['id', 'Age','target','Flight Distrance','Departure Delay in Minutes', 'Arrival Delay in Minutes']):
    test['{}'.format(i)] = test['{}'.format(i)].astype('category')

In [50]:
train.info()
#타입 변형 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   id                                 3000 non-null   int64   
 1   Gender                             3000 non-null   category
 2   Customer Type                      3000 non-null   category
 3   Age                                3000 non-null   int64   
 4   Type of Travel                     3000 non-null   category
 5   Class                              3000 non-null   category
 6   Flight Distance                    3000 non-null   category
 7   Seat comfort                       3000 non-null   category
 8   Departure/Arrival time convenient  3000 non-null   category
 9   Food and drink                     3000 non-null   category
 10  Gate location                      3000 non-null   category
 11  Inflight wifi service              3000 non

In [125]:
# train, valid 데이터 전처리

In [51]:
x=train[train.columns.difference(['id','target'])]

y = train['target']

x_test=test[test.columns.difference(['id','target'])]

x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size = 0.2, random_state = 42)

x_train,  y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

# light gbm model learning

In [89]:
lgbm = LGBMClassifier(n_estimators=1000, num_leaves=50, subsample=0.8,
                      min_child_samples=60, max_depth=20)

In [92]:
lgbm.fit(x_train, y_train,  verbose=True)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=20,
               min_child_samples=60, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=50, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=0.8, subsample_for_bin=200000, subsample_freq=0)

In [96]:
predict_test = lgbm.predict(test[test.columns.difference(['id'])])
# 모델 target 예측값 산출

# 데이콘에 제출할 output 파일을 만들기 위해 dataframe으로 변환한 다음 concat

In [105]:
pd.DataFrame(predict_test)

,0
0,1
1,0
2,1
3,1
4,1
...,...
1995,0
1996,1
1997,0
1998,1


In [112]:
pd.DataFrame(sample['id'])

,id
0,1
1,2
2,3
3,4
4,5
...,...
1995,1996
1996,1997
1997,1998
1998,1999


In [116]:
output=pd.concat([pd.DataFrame(sample['id']), pd.DataFrame(predict_test)],  axis = 1)

In [118]:
output.rename(columns = {0 : 'target'}, inplace = True)
# 컬럼 이름 변경

In [121]:
output.to_csv('data/output.csv', index=False)